In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import numpy as np 
import pandas as pd 
import bz2
import gc
import chardet
import re
import os
import random

# Pre-processing

In [25]:
#Checking files in Kaggle
# List data files that are connected to the kernel
os.listdir('../input')

['test.ft.txt.bz2', 'train.ft.txt.bz2']

In [26]:
os.listdir('../input')

['test.ft.txt.bz2', 'train.ft.txt.bz2']

In [27]:
# Read Train & Test Files

#Kaggle
train_file = bz2.BZ2File('../input/train.ft.txt.bz2')
test_file = bz2.BZ2File('../input/test.ft.txt.bz2')

#Localhost
#train_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/train.ft.txt.bz2')
#test_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/test.ft.txt.bz2')

#Localhost - Versión recortada del archivo
#train_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/Version_Recortada/r_train.ft.txt.bz2')
#test_file = bz2.BZ2File('C:/Users/Lenovo/Documents/GitHub/Datasets/amazonreviews/Version_Recortada/r_test.ft.txt.bz2')

#Create Lists containing Train & Test sentences
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

#Convert from raw binary strings to strings that can be parsed
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

In [28]:
#Delete memory reference (?)
del train_file, test_file
#Garbage collector
gc.collect()

64

In [29]:
print("Cantidad de elementos del Training Set: {}".format(len(train_file_lines)))
print("Cantidad de elementos del Testing Set: {}".format(len(test_file_lines)))

Cantidad de elementos del Training Set: 3600000
Cantidad de elementos del Testing Set: 400000


## Clean data

In [30]:
# Change labels: __label__1 -> 0 (Negative) / __label__2 -> 1 (Positive)
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_lines]

# Make everything Lower Case
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
    
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file_lines]

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

# Modify URLs to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])

## Checking data before and after cleaning

In [31]:
#Random
r = random.randint(1,len(train_file_lines))

#Before
print("Data before cleaning:\n{}".format(train_file_lines[r-1:r]))

#After
print("\nData after cleaning:\n{}".format((train_sentences[r-1:r])))

#Labels
print("\nLabel:{}".format(train_labels[r-1:r]))

Data before cleaning:
["__label__1 HOPE SINKS!!!: This movie was so boring, I almost didn't finish it! It doesn't make you care about the characters and what happens to them. The story had potential but the movie just dragged. I was disappointed that Sandra Bullock and Harry Connick Jr (a potentially great team) would do such a movie. Don't waste your money on this one!\n"]

Data after cleaning:
["hope sinks!!!: this movie was so boring, i almost didn't finish it! it doesn't make you care about the characters and what happens to them. the story had potential but the movie just dragged. i was disappointed that sandra bullock and harry connick jr (a potentially great team) would do such a movie. don't waste your money on this one!"]

Label:[0]


### Output
From the above output it can be seen that each sentence begins with it's sentiment (label1 -> Negative, label2 -> Positive), which is then followed by the review and ends with a newline character \n.

So, first I go convert all the labels to O(Negative) and 1(Positive) and store it in lists that only contain the label values. After this, I store the remainder of the sentence excluding the newline character in lowercase in lists. Also, convert all numbers to 0.


In [32]:
#Delete memory reference (?)
del train_file_lines, test_file_lines
#Garbage collector
gc.collect()

0

## Text Pre-processing

In [33]:
from keras.preprocessing import text, sequence

#Base definitions for text preprocessing
max_features = 20000
maxlen = 100

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [34]:
#Tokenizer definition
#Filtro caracteres especiales usando el Tokenizer de keras.
tokenizer = text.Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')

#Fit on text -> Only the train dataset !!!
tokenizer.fit_on_texts(train_sentences)

#Training set
tokenized_train = tokenizer.texts_to_sequences(train_sentences)
X_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

#Test set
tokenized_test = tokenizer.texts_to_sequences(test_sentences)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [35]:
#Print a random matrix
X_train[r]
# summarize what was learned -> Si quiero ver el tokenizer que aprendio usando los 2 parametros (Max_features,max_length)
#print(t.word_counts)
#print(t.document_count)
#print(t.word_index)
#print(t.word_docs)

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
         837,  1750,     8,     9,    36,  1066,  1598,     1,   106,
           9,   134,     2,    58,    23,    54,   104,  1984,     5,
         120,     6,   921, 12533,   997,    67,   232,  6290,  1393,
          16,    44,    67,   191,  1359,    84,     1,   198,   553,
         200,     9,   274,    56,     3,   140,  4567,  9863,    11,
          90,    97,    13,  9042,    11,    42,  5078,  3176,   296,
         138,   334,     9,   169,    18,     1,   894,     2,     1,
        1010,     9,   232,    32,    16,     8,     9,    36,     4,
          58,    62,    14,    84,    32,    14,    84,   128,    36,
          58], dtype=int32)

### Validation dataset

In [37]:
from sklearn.model_selection import train_test_split
# Create a validation dataset
validation_size = 0.2
X_train, X_valid, train_labels, valid_labes = train_test_split(X_train, train_labels, test_size = validation_size)

In [38]:
#Delete memory reference (?)
del tokenized_test, tokenized_train, tokenizer, train_sentences, test_sentences
#Garbage collector
gc.collect()

159

## Model

In [40]:
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.models import Sequential
from keras.layers import Convolution1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization
from keras.layers.embeddings import Embedding

In [41]:
#Defino los parametros del modelo:
p = 0.25 #Dropout
lr = 0.0001 #Learning Rate
batch_size = 1024
epochs = 10

#Embedding size -> Ver para que sirve, todavia falta entenderlo?
embed_size = 32

In [42]:
# Creo el modelo
model=Sequential()

#Embedding
model.add(Embedding(max_features, embed_size, input_length=maxlen))

#CNN
model.add(Convolution1D(filters=32, kernel_size=2, padding="same", name='Conv1'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

#model.add(Convolution1D(filters=16, kernel_size=3, padding="same", name='Conv2',input_shape=input_shape))
model.add(Convolution1D(filters=16, kernel_size=2, padding="same", name='Conv2'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(MaxPooling1D(pool_size=2, name='MaxPool1'))
model.add(Dropout(p))

model.add(Flatten())

model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(p))

model.add(Dense(1, activation='softmax'))

model.summary()

#Optimizers
ADAM = optimizers.Adam(lr=lr)
model.compile(loss = 'binary_crossentropy', optimizer=ADAM, metrics=['binary_accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           640000    
_________________________________________________________________
Conv1 (Conv1D)               (None, 100, 32)           2080      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 32)           128       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 32)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 32)           0         
_________________________________________________________________
Conv2 (Conv1D)               (None, 100, 16)           1040      
_________________________________________________________________
batch_normalization_2 (Batch (None, 100, 16)           64        
__________

In [43]:
# Callbacks

## Callback para guardar pesos
checkpointer = ModelCheckpoint(filepath='CNN_Sentiment_Analysis_Amazon_Reviews.hdf5', monitor='val_loss'
                                   ,verbose=1, save_best_only=True, mode='min')
callbacks = [checkpointer]

In [44]:
# Fit del modelo
model.fit(X_train
          ,train_labels
          ,epochs=epochs
          ,batch_size = batch_size          
          ,shuffle = True
          ,validation_data = (X_valid,valid_labes)
          #,validation_split=0.20 -> Para no separar en validation antes.
          ,callbacks=callbacks)


Train on 2880000 samples, validate on 720000 samples
Epoch 1/10
2880000/2880000 [==============================] - 98s 34us/step - loss: 7.9756 - binary_accuracy: 0.4997 - val_loss: 7.9534 - val_binary_accuracy: 0.5011

Epoch 00001: val_loss improved from inf to 7.95343, saving model to CNN_Sentiment_Analysis_Amazon_Reviews.hdf5
Epoch 2/10
2880000/2880000 [==============================] - 90s 31us/step - loss: 7.9756 - binary_accuracy: 0.4997 - val_loss: 7.9534 - val_binary_accuracy: 0.5011

Epoch 00002: val_loss did not improve
Epoch 3/10
2880000/2880000 [==============================] - 90s 31us/step - loss: 7.9756 - binary_accuracy: 0.4997 - val_loss: 7.9534 - val_binary_accuracy: 0.5011

Epoch 00003: val_loss did not improve
Epoch 4/10
2880000/2880000 [==============================] - 90s 31us/step - loss: 7.9756 - binary_accuracy: 0.4997 - val_loss: 7.9534 - val_binary_accuracy: 0.5011

Epoch 00004: val_loss did not improve
Epoch 5/10
2880000/2880000 [==========================

## Test 

In [45]:
score, acc = model.evaluate(X_test, test_labels, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

400000/400000 [==============================] - 4s 9us/step
Test score: 7.971192351379394
Test accuracy: 0.5
